In [3]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import pickle

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

objc[46467]: Class CaptureDelegate is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x10fa88860) and /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x28f6f6480). One of the two will be used. Which one is undefined.
objc[46467]: Class CVWindow is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x107904a68) and /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x28f6f64d0). One of the two will be used. Which one is undefined.
objc[46467]: Class CVView is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x107904a90) and /Users/fuixlabsdev1/Programming/PP/graduation-th

### 1. Set up important functions and variables

In [4]:
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE"
]

headers = ["label"] # Label column

for lm in IMPORTANT_LMS:
    headers += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]


def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in IMPORTANT_LMS:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

### 2. Analyze and detection bad pose

Look through [this](./analyze_bad_pose.ipynb) on how we analyze bad feet and knee placement while doing squat.

In [5]:
import math 


def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


def analyze_feet_placement(results, visibility_threshold: int) -> int:
    '''
    Calculate the ratio between the width of 2 knee to the shoulder width

    Return result explanation:
        -1: Unknown result due to poor visibility
        0: Correct knee placement
        1: knee placement too tight
        2: knee placement too wide
    '''

    landmarks = results.pose_landmarks.landmark

    # Analyze visibility
    left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
    right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility

    left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
    right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility

    # If visibility of any keypoints is low cancel the analysis
    if (left_shoulder_vis < visibility_threshold or right_shoulder_vis < visibility_threshold or left_ankle_vis < visibility_threshold or right_ankle_vis < visibility_threshold):
        return -1

    # Calculate shoulder width
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

    shoulder_width = calculate_distance(left_shoulder, right_shoulder)

    # Calculate 2-feet width
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

    feet_width = calculate_distance(left_ankle, right_ankle)

    # Calculate feet and shoulder ratio
    feet_shoulder_ratio = round(feet_width / shoulder_width, 1)
    
    if 1 <= feet_shoulder_ratio <= 1.7:
        return 0
    elif feet_shoulder_ratio < 1:
        return 1
    elif feet_shoulder_ratio > 1.7:
        return 2


def analyze_knee_placement(results, stage: str, visibility_threshold: int) -> int:
    '''
    Calculate the ratio between the width of 2 knee to the foot width

    Return result explanation:
        -1: Unknown result due to poor visibility
        0: Correct knee placement
        1: Knee placement too tight
        2: Knee placement too wide
    '''
    landmarks = results.pose_landmarks.landmark

    # Analyze visibility
    left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
    right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility

    left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
    right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility

    # If visibility of any keypoints is low cancel the analysis
    if (left_ankle_vis < visibility_threshold or right_ankle_vis < visibility_threshold or left_knee_vis < visibility_threshold or right_knee_vis < visibility_threshold):
        return -1

    # Calculate 2-feet width
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

    feet_width = calculate_distance(left_ankle, right_ankle)

    # Calculate 2 knee width
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

    knee_width = calculate_distance(left_knee, right_knee)

    # Calculate feet and shoulder ratio
    knee_feet_ratio = round(knee_width / feet_width, 1)
    
    # Body in UP position
    if stage == "up":
        if 0.6 <= knee_feet_ratio <= 1.2:
            return 0
        elif knee_feet_ratio < 0.6:
            return 1
        elif knee_feet_ratio > 1.2:
            return 2
    # Body in DOWN position
    else:
        if 0.8 <= knee_feet_ratio <= 1.3:
            return 0
        elif knee_feet_ratio < 0.8:
            return 1
        elif knee_feet_ratio > 1.3:
            return 2
    

### 3. Make detection

In [6]:
VIDEO_PATH = "../data/squat/squat_test_1.mov"
VIDEO_PATH2 = "../data/squat/squat_test_2.mov"
VIDEO_PATH3 = "../data/squat/squat_right_2.mp4"

In [7]:
# Load model for counter
with open("./model/squat_model.pkl", "rb") as f:
    count_model = pickle.load(f)

In [14]:
cap = cv2.VideoCapture(VIDEO_PATH)

# Counter vars
counter = 0
current_stage = ""
PREDICTION_PROB_THRESHOLD = 0.7

# Error vars
VISIBILITY_THRESHOLD = 0.7


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        if not ret:
            break

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)
        if not results.pose_landmarks:
            continue

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=2), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=1))

        # Make detection
        try:
            # * Model prediction for SQUAT counter
            # Extract keypoints from frame for the input
            row = extract_important_keypoints(results)
            X = pd.DataFrame([row], columns=headers[1:])

            # Make prediction and its probability
            predicted_class = count_model.predict(X)[0]
            prediction_probabilities = count_model.predict_proba(X)[0]
            prediction_probability = round(prediction_probabilities[prediction_probabilities.argmax()], 2)

            # Evaluate model prediction
            if predicted_class == "down" and prediction_probability >= PREDICTION_PROB_THRESHOLD:
                current_stage = "down"
            elif current_stage == "down" and predicted_class == "up" and prediction_probability >= PREDICTION_PROB_THRESHOLD: 
                current_stage = "up"
                counter += 1
            
            # * Evaluate FEET PLACEMENT error
            feet_placement_evaluation = analyze_feet_placement(results, VISIBILITY_THRESHOLD)

            if feet_placement_evaluation == -1:
                feet_placement = "UNK"
            elif feet_placement_evaluation == 0:
                feet_placement = "Correct"
            elif feet_placement_evaluation == 1:
                feet_placement = "Too tight"
            elif feet_placement_evaluation == 2:
                feet_placement = "Too wide"
            
            # * Evaluate KNEE PLACEMENT error
            knee_placement_evaluation = analyze_knee_placement(results, current_stage, VISIBILITY_THRESHOLD)

            if knee_placement_evaluation == -1:
                knee_placement = "UNK"
            elif knee_placement_evaluation == 0:
                knee_placement = "Correct"
            elif knee_placement_evaluation == 1:
                knee_placement = "Too tight"
            elif knee_placement_evaluation == 2:
                knee_placement = "Too wide"
            
            # Visualization
            # Status box
            cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

            # Display class
            cv2.putText(image, "COUNT", (10, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, f'{str(counter)}, {predicted_class.split(" ")[0]}, {str(prediction_probability)}', (5, 40), cv2.FONT_HERSHEY_COMPLEX, .7, (255, 255, 255), 2, cv2.LINE_AA)

            # Display Feet and Shoulder width ratio
            cv2.putText(image, "FEET", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, feet_placement, (195, 40), cv2.FONT_HERSHEY_COMPLEX, .7, (255, 255, 255), 2, cv2.LINE_AA)

            # Display knee and Shoulder width ratio
            cv2.putText(image, "KNEE", (330, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, knee_placement, (325, 40), cv2.FONT_HERSHEY_COMPLEX, .7, (255, 255, 255), 2, cv2.LINE_AA)

        except Exception as e:
            print(f"Error: {e}")
        
        # Reduce size of a frame
        image = rescale_frame(image, 60)

        cv2.imshow("CV2", image)
        
        # Press Q to close cv2 window
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range (1, 5):
        cv2.waitKey(1)
  